# IVSCC Morphology Data Analysis Pipeline

# Author: Xiaoxiao Liu,  xiaoxiaol@alleninstitute.org
## Date: 10/31/2015

In [5]:
import pandas as pd
import os
import platform
import sys



# Module PATHs
if (platform.system() == "Linux"):
        WORK_PATH = "/local1/xiaoxiaol/work"
else:
        WORK_PATH = "/Users/xiaoxiaoliu/work"
        
        
        
p =  WORK_PATH + '/src/cell-type-analysis'
sys.path.append(p)

sys.path.append(p + '/utilities')
import morph_nfb_2_csv as nfb

sys.path.append(p + '/blast_neuron')
import blast_neuron_comp as bn

        
        
# Data PATHs     
data_dir = '/local1/xiaoxiaol/work/data/lims2/1027_pw_aligned'
csv_file = data_dir + "/pw_aligned.csv"  ## where the transform parameters are, obtained from lims
origin_data_DIR = data_dir + "/original"

###################################
# Preprocessing SWC files (alignment)
###################################

In [ ]:
# Pull Data From Lims  
##  run sql to grab both alignment paraemters and swc file locations and associated meta info
### pia_alignment_transforms.sql
### save to pw_aligned.csv
### To obtain available swc files and corresponding database tags ( dendrite_type, layer info) :
### query_nr_va_ephys_qc.sql
### save to ephys_qc_filtered.csv

### Review spreadsheets and fix db erros (missing alignment parameters, etc.)

In [5]:
# Download/copy swc files 

## python ../utilities/downloadSWC_filter.py  -i ~/work/data/lims2/0923_pw_aligned/0923_pw_aligned.csv	 -o ~/work/data/lims2/0923_pw_aligned/original

In [ ]:
# Apply Pia-WhiteMatter Alignments
import applyPiaTransformToSWCs as pwTrans


transform_DIR = data_dir + "/transforms"  ## where to store the transform.txt
output_DIR = data_dir + "/pw_aligned"



## generate transform txt files
if not os.path.exists(transform_DIR):
    os.makedirs(transform_DIR)

pwTrans.generateTransformFilesfromCSV(transform_DIR, csv_file)

## apply transforms to swc files
if not os.path.exists(output_DIR):
    os.makedirs(output_DIR)

df = pd.read_csv(csv_file)
df = df[df.specimen_id != 464326095]  # this specimenid's alignments are wrong
data_table = df.values
num_samples, num_cols = data_table.shape


for i in range(num_samples):
    orca_path = data_table[i][num_cols - 1]
    fn = orca_path.split('/')[-1]
    transform_fn = transform_DIR + "/%s.txt" % fn.split('.swc')[0]
    output_fn = output_DIR + "/" + fn
    pwTrans.applyTransformBySpecimenName(origin_data_DIR + '/' + fn, transform_fn, output_fn)
    
bn.genLinkerFile(output_DIR, output_DIR+"/mylinker.ano")

In [22]:
# Remove Axons
import remove_axons as ra


df = pd.read_csv(data_dir +'/pw_aligned/mylinker.ano', header=None)

output_dir = data_dir +'/axon_removed'


if not os.path.exists(output_dir):
        os.mkdir(output_dir)

for i in range(df.shape[0]):
     swc_file = df.iloc[i,0]
     swc_file= swc_file[8:]
     output_swc_file = output_dir +'/'+swc_file.split('/')[-1]
     #print output_swc_file
     ra.remove_swc_by_id(2, swc_file, output_swc_file)


/local1/xiaoxiaol/work/data/lims2/1027_pw_aligned/axon_removed/Cux2-CreERT2_Ai14-198017.05.01.01_488497374_m.swc
/local1/xiaoxiaol/work/data/lims2/1027_pw_aligned/axon_removed/Gad2-IRES-Cre_Ai14_IVSCC_-172679.03.01.01_471076778_m.swc
/local1/xiaoxiaol/work/data/lims2/1027_pw_aligned/axon_removed/Gad2-IRES-Cre_Ai14_IVSCC_-172679.05.01.01_485247448_m.swc
/local1/xiaoxiaol/work/data/lims2/1027_pw_aligned/axon_removed/Gad2-IRES-Cre_Ai14_IVSCC_-177637.02.02.01_475124390_m.swc
/local1/xiaoxiaol/work/data/lims2/1027_pw_aligned/axon_removed/Htr3a-Cre_NO152_Ai14-175479.06.02.01_488465816_m.swc
/local1/xiaoxiaol/work/data/lims2/1027_pw_aligned/axon_removed/Htr3a-Cre_NO152_Ai14_IVSCC_-175481.02.02.01_475459187_m.swc
/local1/xiaoxiaol/work/data/lims2/1027_pw_aligned/axon_removed/Htr3a-Cre_NO152_Ai14_IVSCC_-175482.03.02.01_475124358_m.swc
/local1/xiaoxiaol/work/data/lims2/1027_pw_aligned/axon_removed/Htr3a-Cre_NO152_Ai14_IVSCC_-177968.02.01.01_488423071_m.swc
/local1/xiaoxiaol/work/data/lims2/1027_

###########################################
# Feature Calculation
###########################################

In [24]:
# Feature Calculation  ( TAKES A WHILE)
import run_feature_calculation as feacal

input_dir = data_dir + "/axon_removed"
preprocessed_dir = data_dir + "/preprocessed"
db_tags_csv_file = data_dir + '/ephys_qc_filtered.csv' 

    
    
## run blast neuron  features
feacal.cal_bn_features(input_dir,preprocessed_dir)

## clean up db tags
## remove alignment outliers?
df_db_tags= feacal.cleanup_query_csv(db_tags_csv_file)

## merge all info
df_features = pd.read_csv(preprocessed_dir + '/features_with_tags.csv')

swc_file_names2 = []
for i in range(df_features.shape[0]):
        swc_fn = df_features.swc_file[i].split('/')[-1]
        swc_file_names2.append(swc_fn)
df_features['swc_file_name'] = pd.Series(swc_file_names2)

merged = pd.merge(df_db_tags, df_features, how='inner', on=['swc_file_name'])
merged.drop(['orca_path', 'swc_file_name', 'region_info'], axis=1, inplace=True)


# add three more tags
# modify height width depth

merged.to_csv(preprocessed_dir + '/features_with_db_tags.csv', index=False)


/local1/xiaoxiaol/work/data/lims2/1027_pw_aligned/axon_removed/Cux2-CreERT2_Ai14-198017.05.01.01_488497374_m.swc
/local1/xiaoxiaol/work/v3d/v3d_external/bin/vaa3d -x resample_swc -f resample_swc -i /local1/xiaoxiaol/work/data/lims2/1027_pw_aligned/axon_removed/Cux2-CreERT2_Ai14-198017.05.01.01_488497374_m.swc -o /local1/xiaoxiaol/work/data/lims2/1027_pw_aligned/preprocessed/Cux2-CreERT2_Ai14-198017.05.01.01_488497374_m.swc -p 1 
Thread started
Thread finished
0
/local1/xiaoxiaol/work/data/lims2/1027_pw_aligned/axon_removed/Gad2-IRES-Cre_Ai14_IVSCC_-172679.03.01.01_471076778_m.swc
/local1/xiaoxiaol/work/v3d/v3d_external/bin/vaa3d -x resample_swc -f resample_swc -i /local1/xiaoxiaol/work/data/lims2/1027_pw_aligned/axon_removed/Gad2-IRES-Cre_Ai14_IVSCC_-172679.03.01.01_471076778_m.swc -o /local1/xiaoxiaol/work/data/lims2/1027_pw_aligned/preprocessed/Gad2-IRES-Cre_Ai14_IVSCC_-172679.03.01.01_471076778_m.swc -p 1 
Thread started
Thread finished
0
/local1/xiaoxiaol/work/data/lims2/1027_pw_al

In [26]:
## Add extra features

fnames = [u'Unnamed: 0', u'specimen_id', u'specimen_name', u'dendrite_type',
       u'cre_line', u'layer', u'swc_file', u'num_nodes', u'soma_surface',
       u'num_stems', u'num_bifurcations', u'num_branches', u'num_of_tips',
       u'overall_depth', u'overall_width', u'overall_height',
       u'average_diameter', u'total_length', u'total_surface', u'total_volume',
       u'max_euclidean_distance', u'max_path_distance', u'max_branch_order',
       u'average_contraction', u'average fragmentation',
       u'parent_daughter_ratio', u'bifurcation_angle_local',
       u'bifurcation_angle_remote', u'moment1', u'moment2', u'moment3',
       u'moment4', u'moment5', u'moment6', u'moment7', u'moment8', u'moment9',
       u'moment10', u'moment11', u'moment12', u'moment13', u'avgR']

df_f = pd.read_csv(preprocessed_dir + '/features_with_db_tags.csv')

df_f.columns = fnames

df_f['height_width_ratio'] = df_f['overall_height']/df_f['overall_width']

df_f['average_branch_length'] = df_f['total_length']/df_f['num_branches']

df_f ['length_surface_ratio'] = df_f ['total_length']/df_f['total_surface']


df_f.to_csv(preprocessed_dir + '/features_with_db_tags_added.csv',index=False)

######################################################
# Data Curation 
######################################################

In [ ]:
# Curate data for clustering

In [35]:
# merge meta info
import merge_meta_info as mmi
import numpy as np


all_feature_file = data_dir + '/preprocessed/features_with_db_tags_added.csv'
output_dir = data_dir + '/clustering_results'

#Meta_CSV_FILE = data_dir + '/IVSCC_qual_calls_XiaoXiao_150cells_092915-UPDATED3.csv'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)




gl_feature_names = np.array(
        ['num_nodes', 'soma_surface', 'num_stems', 'num_bifurcations', 'num_branches', 'num_of_tips',
         'overall_width', 'overall_height', 'overall_depth', 'average_diameter', 'total_length',
         'total_surface', 'total_volume', 'max_euclidean_distance', 'max_path_distance', 'max_branch_order',
         'average_contraction', 'average fragmentation', 'parent_daughter_ratio', 'bifurcation_angle_local',
         'bifurcation_angle_remote','height_width_ratio','average_branch_length','length_surface_ratio'])

# remove scales
gl_feature_names_inv = np.array(
    ['num_nodes', 'soma_surface', 'num_stems', 'num_bifurcations', 'num_branches', 'num_of_tips',
     'average_diameter', 'total_length',
     'total_surface', 'total_volume', 'max_euclidean_distance', 'max_path_distance', 'max_branch_order',
     'average_contraction', 'average fragmentation', 'parent_daughter_ratio', 'bifurcation_angle_local',
     'bifurcation_angle_remote'])

gmi_feature_names = np.array(
    ['moment1', 'moment2', 'moment3', 'moment4', 'moment5', 'moment6', 'moment7', 'moment8',
     'moment9', 'moment10', 'moment11', 'moment12', 'moment13'])  ### removed ave_R

## generate creline specific anos
mmi.generateLinkerFileFromCSV(preprocessed_dir, all_feature_file ,"cre_line",False)



In [36]:
# MAUNUAL : Identify Outliers ( Alignment errors, db logging erros)
## save to wrong_scale.csv
to_remove_filename = data_dir +'/wrong_scale_to_remove.csv'

In [37]:

# require the following col names in the merged spread sheet
col_names = ['specimen_id','specimen_name','cre_line','layer','dendrite_type','swc_file']#,'types']
all_feature_names = np.append(gl_feature_names, gmi_feature_names)
col_names.extend(all_feature_names)


# remove outliers identified in
df_remove  = pd.read_csv(to_remove_filename)
df_complete = pd.read_csv(all_feature_file)
df_complete_filter = df_complete[~df_complete['specimen_name'].isin(df_remove['specimen_name'])]


# !!!!! df_complete_filter drop  dendrite_type and layer

#df_meta = pd.read_csv(Meta_CSV_FILE)
#merged = pd.merge(df_complete_filter,df_meta,how='inner',on=['specimen_name'])

merged = df_complete_filter
merged = merged[col_names]
merged[all_feature_names] = merged[all_feature_names].astype(float)


output_merged_csv = data_dir +'/meta_merged_allFeatures.csv'
merged.to_csv(output_merged_csv,index=False)

#generateLinkerFileFromCSV(output_dir, output_merged_csv,'cre_line',False)

mmi.generateLinkerFileFromCSV(output_dir, output_merged_csv,None,False)

#output_dir= data_DIR + '/figures/staci_types'
#generateLinkerFileFromCSV(output_dir, output_merged_csv,'types',False)
#swc_screenshot_folder =  data_dir + "/figures/pw_aligned_bmps"
#types = np.unique(merged['types'])
#for type in types:
#    print type
#    df_type = merged[merged.types == type]
#    type_format_string = '_'.join(type.split(" "))
#    mmi.copySnapshots(df_type, swc_screenshot_folder, output_dir + '/' +type_format_string )
#    mmi.assemble_screenshots(output_dir + '/' + type_format_string, output_dir + '/' +type_format_string+ '_assemble.png', 128)


################################################
# Clustering and Visualizations
################################################

In [ ]:
#